# Change-Point Detection in Business Cycles

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt


In [2]:
# These are exchange rate data's, indexed, not actual values
neer_data = pd.read_excel('../data/processed_data/georgia_neer_data_cleaned_processed.xlsx')
reer_data = pd.read_excel('../data/processed_data/georgia_reer_data_cleaned_processed.xlsx')
# Date is in the format of MM-YYYY, must be converted to YYYY-MM-DD, where day is the first day of the month
neer_data['Date'] = pd.to_datetime(neer_data['Date'])
reer_data['Date'] = pd.to_datetime(reer_data['Date'])

# This is GDP data

gdp_data = pd.read_excel('../data/processed_data/georgia_monthly_gdp_chow_lin.xlsx')


#We must add neer and reer to the gdp data, by date.

# Merge NEER and REER data with GDP data
gdp_data = gdp_data.merge(neer_data, on='Date', how='left')
gdp_data = gdp_data.merge(reer_data, on='Date', how='left')

/var/folders/50/97r9rj4j6s9f_0_j75x32sxr0000gn/T/ipykernel_80995/3149601007.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  neer_data['Date'] = pd.to_datetime(neer_data['Date'])
/var/folders/50/97r9rj4j6s9f_0_j75x32sxr0000gn/T/ipykernel_80995/3149601007.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  reer_data['Date'] = pd.to_datetime(reer_data['Date'])


In [3]:
neer_data

,Date,NEER (Dec-2010=100)
0,2011-01-01,98.445895
1,2011-02-01,99.346752
2,2011-03-01,102.133186
3,2011-04-01,103.322941
4,2011-05-01,103.443282
...,...,...
167,2024-12-01,184.967050
168,2025-01-01,183.932411
169,2025-02-01,182.899682
170,2025-03-01,182.457123


In [4]:
gdp_data

,Date,YoY Growth (%),Inflation_Rate,Real_GDP_Growth,q_str,GDP_at_market_prices,GDP per capita in GEL,"GDP per capita, USD",GDP in mil. USD,GDP_at_market_prices_monthly,GDP per capita in GEL_monthly,"GDP per capita, USD_monthly",GDP in mil. USD_monthly,NEER (Dec-2010=100),Real Effective Exchange Rate - Index (Dec-10=100),GEL/USD Real Exchange Rate - Index (Dec-10=100),GEL/EUR Real Exchange Rate - Index (Dec-10=100),GEL/TRY Real Exchange Rate - Index (Dec-10=100),GEL/RUB Real Exchange Rate - Index (Dec-10=100)
0,2011-01-01,9.6,2.0537,7.394440,2011Q1,5420.832044,1443.076775,819.517640,3078.469255,1809.399484,481.841744,264.603425,993.830608,98.445895,99.790539,99.720145,99.759842,102.612835,95.450989
1,2011-02-01,4.4,2.8186,1.538049,2011Q1,5420.832044,1443.076775,819.517640,3078.469255,1731.745438,460.888141,264.739568,994.565127,99.346752,102.541156,103.170221,101.269449,108.108247,95.835017
2,2011-03-01,3.7,-0.0218,3.722612,2011Q1,5420.832044,1443.076775,819.517640,3078.469255,1879.687122,500.346890,290.174647,1090.073520,102.133186,104.569463,105.961545,100.980795,110.936535,95.915597
3,2011-04-01,6.0,-0.1427,6.151478,2011Q2,6383.402398,1699.322110,1020.357974,3832.914019,2073.909071,552.125576,325.392294,1222.339876,103.322941,105.087841,108.102531,100.042467,108.626196,97.043941
4,2011-05-01,1.7,0.3750,1.320050,2011Q2,6383.402398,1699.322110,1020.357974,3832.914019,2052.741504,546.266563,335.710190,1261.240542,103.443282,104.810420,107.304211,100.022039,109.262619,95.369141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2024-08-01,12.0,-0.0866,12.097076,2024Q3,24855.693948,6727.559175,2479.082588,9159.238363,8321.491062,2252.341894,831.579201,3072.354486,184.227682,117.104860,76.046501,97.908365,179.625378,124.990165
164,2024-09-01,8.3,0.2868,7.990284,2024Q3,24855.693948,6727.559175,2479.082588,9159.238363,8339.663237,2257.493074,831.275016,3070.878538,184.077348,116.550510,75.911753,96.959458,176.451175,127.634442
165,2024-10-01,11.0,0.2765,10.693931,2024Q4,25309.564202,6850.405835,2483.640106,9176.076604,8468.356442,2291.994089,834.293487,3082.535272,185.250622,116.664495,75.514775,97.929356,171.819969,132.746402
166,2024-11-01,7.5,0.7082,6.744039,2024Q4,25309.564202,6850.405835,2483.640106,9176.076604,8420.097681,2279.056979,825.343899,3049.267964,187.620773,118.146113,75.842423,101.173307,169.644465,137.301783
